In [1]:
import os
import copy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import helpers

In [3]:
merged_metadata, relatedness, genotypes_array, mapping_info = helpers.load_data()

In [49]:
from scipy.spatial.distance import pdist
from scipy.stats import spearmanr
import numpy as np
from sklearn.metrics.pairwise import haversine_distances

def haversine_vectorized(coords):
    """
    Compute pairwise haversine distances in a vectorized manner.
    coords: (n_samples, 2) array of [latitude, longitude] in radians.
    Returns:
        Condensed distance matrix (pdist format).
    """
    lat = coords[:, 0][:, np.newaxis]  # Reshape for broadcasting
    lon = coords[:, 1][:, np.newaxis]

    # Compute differences
    dlat = lat - lat.T
    dlon = lon - lon.T

    # Haversine formula
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat) * np.cos(lat.T) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Earth's radius (6371 km)
    distances = 6371 * c

    return distances

def preservation_metric(gt_dists, ac_dists, num_dists=50000):
    """
    Compute the Spearman correlation between two pairwise distance matrices.
    gt_dists: Ground truth condensed distance matrix.
    ac_dists: Ancestry condensed distance matrix.
    num_samples: Number of distances to sample for Spearman correlation.
    """
    # Take a random subset of distances for Spearman correlation
    subset = np.random.choice(len(ac_dists), min(num_dists, len(ac_dists)), replace=False)

    # Compute Spearman correlation
    corr, _ = spearmanr(gt_dists, ac_dists, axis=0)
    return corr


def compute_metrics(ancestry_coords, merged_metadata):
    include_index = merged_metadata['Genetic_region'] != 'AMR'
    ground_truth_coords = merged_metadata[['latitude', 'longitude']]
    ground_truth_coords = np.radians(ground_truth_coords)
    
    ancestry_dists = pdist(ancestry_coords)
    ground_truth_coords_rad = np.radians(ground_truth_coords[include_index])
    ground_truth_dists = haversine_vectorized(ground_truth_coords_rad.values)

    geographic_preservation = preservation_metric(ground_truth_dists, 
                                                  ancestry_dists)

    admix_ratios = pd.read_csv('/lustre06/project/6065672/sciclun4/ActiveProjects/phate_genetics/data/1000G/1000G_admix_ratios_inc_EURAFR')
    admix_ratios = admix_ratios.set_index('Unnamed: 0')
    merged_metadata2 = pd.merge(merged_metadata, admix_ratios, left_index=True, right_index=True, how='left')

    amr_idx = (merged_metadata2['Project'] == '1000_Genomes') & (merged_metadata2['Genetic_region'] == 'AMR')
    eur_idx = (merged_metadata2['Project'] == '1000_Genomes') & (merged_metadata2['Genetic_region'] == 'EUR')
    afr_idx = (merged_metadata2['Project'] == '1000_Genomes') & (merged_metadata2['Genetic_region'] == 'AFR')

    merged_metadata2.loc[eur_idx, 
                     'European ancestry (%)'] = 1.0
    merged_metadata2.loc[afr_idx & (merged_metadata2['Population'].isin(['GWD', 'ESN', 'YRI', 'LWK', 'MSL'])), 
                     'African ancestry (%)'] = 1.0
    
    include_index2 = (merged_metadata2['Project'] == '1000_Genomes') & merged_metadata2['Genetic_region'].isin(['AMR', 'AFR', 'EUR']) 
    include_index2 = include_index2 & ~np.isnan(merged_metadata2[include_index2]['African ancestry (%)']) & ~np.isnan(merged_metadata2[include_index2]['European ancestry (%)']) & ~np.isnan(merged_metadata2[include_index2]['Amerindigenous ancestry (%)'])
    admix_coords = merged_metadata2[['African ancestry (%)', 'Amerindigenous ancestry (%)', 'European ancestry (%)']][include_index2]

    admixture_dists = pdist(admix_coords[include_index2])
    
    admixture_preservation = preservation_metric(admixture_dists, 
                                                 ancestry_dists)
    return geographic_preservation, admixture_preservation

In [64]:
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr
import numpy as np
from sklearn.metrics.pairwise import haversine_distances

def haversine_vectorized(coords):
    """
    Compute pairwise haversine distances in a vectorized manner.
    coords: (n_samples, 2) array of [latitude, longitude] in radians.
    Returns:
        Square pairwise distance matrix.
    """
    lat = coords[:, 0][:, np.newaxis]  # Reshape for broadcasting
    lon = coords[:, 1][:, np.newaxis]

    # Compute differences
    dlat = lat - lat.T
    dlon = lon - lon.T

    # Haversine formula
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat) * np.cos(lat.T) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Earth's radius (6371 km)
    distances = 6371 * c

    return distances

def preservation_metric(gt_dists, ac_dists, num_dists=50000):
    """
    Compute the Spearman correlation between two pairwise distance matrices.
    gt_dists: Ground truth condensed distance matrix.
    ac_dists: Ancestry condensed distance matrix.
    num_samples: Number of distances to sample for Spearman correlation.
    """
    # Take a random subset of distances for Spearman correlation
    subset = np.random.choice(len(ac_dists), min(num_dists, len(ac_dists)), replace=False)

    # Compute Spearman correlation
    corr, _ = spearmanr(gt_dists[subset], ac_dists[subset], axis=0)
    return corr

def compute_metrics(ancestry_coords, merged_metadata):
    include_index = merged_metadata['Genetic_region'] != 'AMR'
    ground_truth_coords = merged_metadata[['latitude', 'longitude']]
    ground_truth_coords = np.radians(ground_truth_coords)
    
    ancestry_dists = pdist(ancestry_coords[include_index])
    ground_truth_coords_rad = np.radians(ground_truth_coords[include_index])
    ground_truth_dists_square = haversine_vectorized(ground_truth_coords_rad.values)
    ground_truth_dists = squareform(ground_truth_dists_square)

    geographic_preservation = preservation_metric(ground_truth_dists, 
                                                  ancestry_dists)

    admix_ratios = pd.read_csv('/lustre06/project/6065672/sciclun4/ActiveProjects/phate_genetics/data/1000G/1000G_admix_ratios_inc_EURAFR')
    admix_ratios = admix_ratios.set_index('Unnamed: 0')
    merged_metadata2 = pd.merge(merged_metadata, admix_ratios, left_index=True, right_index=True, how='left')

    amr_idx = (merged_metadata2['Project'] == '1000_Genomes') & (merged_metadata2['Genetic_region'] == 'AMR')
    eur_idx = (merged_metadata2['Project'] == '1000_Genomes') & (merged_metadata2['Genetic_region'] == 'EUR')
    afr_idx = (merged_metadata2['Project'] == '1000_Genomes') & (merged_metadata2['Genetic_region'] == 'AFR')

    merged_metadata2.loc[eur_idx, 
                     'European ancestry (%)'] = 1.0
    merged_metadata2.loc[afr_idx & (merged_metadata2['Population'].isin(['GWD', 'ESN', 'YRI', 'LWK', 'MSL'])), 
                     'African ancestry (%)'] = 1.0
    
    include_index2 = (merged_metadata2['Project'] == '1000_Genomes') & merged_metadata2['Genetic_region'].isin(['AMR', 'AFR', 'EUR']) 
    include_index2 = include_index2 & ~np.isnan(merged_metadata2[include_index2]['African ancestry (%)']) & ~np.isnan(merged_metadata2[include_index2]['European ancestry (%)']) & ~np.isnan(merged_metadata2[include_index2]['Amerindigenous ancestry (%)'])
    admix_coords = merged_metadata2[['African ancestry (%)', 'Amerindigenous ancestry (%)', 'European ancestry (%)']][include_index2]

    admixture_dists = pdist(admix_coords)
    ancestry_dists = pdist(ancestry_coords[include_index2])
    admixture_preservation = preservation_metric(admixture_dists, 
                                                 ancestry_dists)
    return geographic_preservation, admixture_preservation

In [5]:
# Step 0: Pre-process data
normalized_matrix, overlap_counts = helpers.preprocess_data_matrix(genotypes_array)

Loading previously computed non-missing overlap matrix...


In [6]:
# Fit PCA model on unrelated samples
filters = ["filter_pca_outlier", "hard_filtered", "filter_contaminated"]
_filtered_indices = merged_metadata[merged_metadata[filters].any(axis=1)].index
filtered_indices = ~merged_metadata.index.isin(_filtered_indices)
related_indices = ~merged_metadata['filter_king_related'].values #np.ones(shape=genotypes_array.shape[0], dtype=bool)

to_fit_on = related_indices & filtered_indices
to_transform_on = (~related_indices) & filtered_indices

In [7]:
pca_emb, pca_obj = helpers.compute_pca_from_data_matrix(normalized_matrix,
                                                         to_fit_on,
                                                         to_transform_on,
                                                         n_components=50)
phate_emb = helpers.compute_phate(pca_emb, to_fit_on, to_transform_on, knn=5, t=5)

Running PHATE on 3400 observations and 50 variables.
Calculating graph and diffusion operator...
  Calculating KNN search...
  Calculated KNN search in 0.27 seconds.
  Calculating affinities...
  Calculated affinities in 0.17 seconds.
Calculated graph and diffusion operator in 0.48 seconds.
Calculating landmark operator...
  Calculating SVD...
  Calculated SVD in 0.16 seconds.
  Calculating KMeans...
  Calculated KMeans in 1.18 seconds.
Calculated landmark operator in 2.03 seconds.
Calculating diffusion potential...
Calculated diffusion potential in 0.85 seconds.
Calculating metric MDS...
Calculated metric MDS in 3.61 seconds.
Calculating KNN search...
Calculated KNN search in 0.05 seconds.
Calculating affinities...


/lustre06/project/6065672/sciclun4/Envs/phate_env/lib/python3.11/site-packages/phate/phate.py:884: RuntimeWarning: Pre-fit PHATE should not be used to transform a new data matrix. Please fit PHATE to the new data by running 'fit' with the new data.
  warnings.warn(


In [65]:
print(compute_metrics(pca_emb, merged_metadata))
print(compute_metrics(pca_emb[:,:2], merged_metadata))
print(compute_metrics(phate_emb, merged_metadata))

(0.5047043566630256, 0.9166183707409576)
(0.5204353682198941, 0.9530824502159497)
(0.4659688456319123, 0.7716861495393533)


In [11]:
from sklearn.random_projection import GaussianRandomProjection
projector = GaussianRandomProjection(n_components=2000, random_state=42)
reduced_genotype_matrix = projector.fit_transform(normalized_matrix)
pca_embr, pca_objr = helpers.compute_pca_from_data_matrix(normalized_matrix,
                                                         to_fit_on,
                                                         to_transform_on,
                                                         n_components=50)
phate_embr = helpers.compute_phate(pca_embr, to_fit_on, to_transform_on, knn=5, t=5)

Running PHATE on 3400 observations and 50 variables.
Calculating graph and diffusion operator...
  Calculating KNN search...
  Calculated KNN search in 0.27 seconds.
  Calculating affinities...
  Calculated affinities in 0.02 seconds.
Calculated graph and diffusion operator in 0.29 seconds.
Calculating landmark operator...
  Calculating SVD...
  Calculated SVD in 0.14 seconds.
  Calculating KMeans...
  Calculated KMeans in 1.02 seconds.
Calculated landmark operator in 1.85 seconds.
Calculating diffusion potential...
Calculated diffusion potential in 0.88 seconds.
Calculating metric MDS...
Calculated metric MDS in 3.71 seconds.
Calculating KNN search...
Calculated KNN search in 0.05 seconds.
Calculating affinities...


/lustre06/project/6065672/sciclun4/Envs/phate_env/lib/python3.11/site-packages/phate/phate.py:884: RuntimeWarning: Pre-fit PHATE should not be used to transform a new data matrix. Please fit PHATE to the new data by running 'fit' with the new data.
  warnings.warn(


In [66]:
print(compute_metrics(pca_embr, merged_metadata))
print(compute_metrics(pca_embr[:,:2], merged_metadata))
print(compute_metrics(phate_embr, merged_metadata))

(0.5010599269225768, 0.9149986754267593)
(0.513880167974442, 0.9532663187402514)
(0.29407077018656425, 0.7841554511006305)


In [67]:
print(compute_metrics(np.random.uniform(size=[pca_embr.shape[0], 10]), 
                      merged_metadata))

(-0.0004689402284907093, -0.0006986268452863316)
